In [7]:
from math import sqrt, pi, cos, sin, radians, atan2, asin
import random
import numpy as np
import pickle
import gurobipy as gp
from gurobipy import GRB
import time

In [8]:
def pos2dist(point1,point2):
    radius = 3958.756;
    lat1 = point1[0]*pi/180
    lat2 = point2[0]*pi/180
    lon1 = point1[1]*pi/180
    lon2 = point2[1]*pi/180
    deltaLat = lat2-lat1;
    deltaLon = lon2-lon1;
    a = sin((deltaLat)/2)**2 + cos(lat1)*cos(lat2) * sin(deltaLon/2)**2
    c = 2*atan2(sqrt(a),sqrt(1-a))
    x = deltaLon*cos((lat1+lat2)/2)
    y = deltaLat
    d1 = radius*c
    d2 = radius*sqrt(x*x + y*y)
    return d1


def separate_quadrant(ml, step):
    q1, q2, q3 = [], [], []
    ix1, ix2, ix3 = [], [], []
    coord_x, coord_y = {}, {}
    x0radi = x0 * pi / 180
    y0radi = y0 * pi / 180
    for i in range(3):
        coord_x[i], coord_y[i] = [], []
    for i in range(nCusts):
        lat1 = ml[i][0] * pi / 180
        lon1 = ml[i][1] * pi / 180
        if  -pi/3 - pi/15 * step <= atan2(lon1-y0radi, lat1-x0radi) < pi/3 - pi/15 * step:
            q1.append( ml[i])
            coord_x[0].append( ml[i][0])
            coord_y[0].append( ml[i][1])
            ix1.append(i)
        elif   -pi  <= atan2(lon1-y0radi, lat1-x0radi) < -pi/3 - pi/15 * step \
        or  pi - pi/15 * step  <= atan2(lon1-y0radi, lat1-x0radi) < pi:
            q2.append(ml[i])
            coord_x[1].append( ml[i][0])
            coord_y[1].append( ml[i][1])
            ix2.append(i)
        else:
            q3.append(ml[i])
            coord_x[2].append( ml[i][0])
            coord_y[2].append( ml[i][1])
            ix3.append(i)
    return q1, q2, q3, ix1, ix2, ix3, coord_x, coord_y


def inner_function(q1, q2, q3, ix, Prize_all, coord_x, coord_y):
    tmp_vec = [q1, q2, q3]
    inner = {}
    inner_real = {}
    inner_virtual = {}
    outer = {}
    Prize = {}
    for i in range(3):
        inner[i] = []
        inner_real[i], inner_virtual[i] = [], []
        outer[i] = []
        Prize[i] = []
        tmp = tmp_vec[i]
        cnt_inner = len(tmp)
        for j in range(len(tmp)):
            if pos2dist((x0,y0), tmp[j]) <= 1.4:
                inner_real[i].append(j)
                inner_virtual[i].append(cnt_inner)
                coord_x[i].append(tmp[j][0])
                coord_y[i].append(tmp[j][1])
                Prize[i].append(1)
                cnt_inner += 1
                ix[i].append(ix[i][j])
            else:
                outer[i].append(j)
                Prize[i].append(Prize_all[ix[i][j]])
        inner[i] = inner_real[i] + inner_virtual[i]
        Prize[i] += [1] * len(inner_virtual[i])
    return inner, inner_real, outer, coord_x, coord_y, Prize, ix

In [9]:
def distance_matrix_quadrant(coord_x_set, quadrant_index, distance_matrix_all):
    N = len(coord_x_set)
    dist_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i != j:
                dist_matrix[i][j] = distance_matrix_all[quadrant_index[i]][quadrant_index[j]]
    return dist_matrix

def subtourelim(model, where):
    if where == GRB.Callback.MIPNODE:
        depth = model.cbGet(GRB.Callback.MIPNODE_NODCNT)
        if depth == 0:
            if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.Status.OPTIMAL:
                valx = model.cbGetNodeRel(model._varsx)
                valy = model.cbGetNodeRel(model._varsy)
                M = mx
                m1 = gp.Model("m1")
                m1.setParam('OutputFlag', 0)  # change to 1 to show Gurobi solver output
                w = m1.addVars(M, M, vtype=GRB.BINARY, name="w")
                z = m1.addVars(M, vtype=GRB.BINARY, name="z")
                zz = m1.addVars(M, vtype=GRB.BINARY, name="zz")
                m1.setObjective(sum(valx[i, j] * w[i, j] for i in range(M) for j in range(M)) - sum(valy[i] * z[i]
                                                                                                    for i in
                                                                                                    range(M)) + sum(
                    valy[i] * zz[i] for i in range(M)), GRB.MAXIMIZE)
                m1.addConstrs(w[i, j] <= z[i] for i in range(M) for j in range(M))
                m1.addConstrs(w[i, j] <= z[j] for i in range(M) for j in range(M))
                m1.addConstrs(zz[i] <= z[i] for i in range(M))
                m1.addConstr(sum(zz[i] for i in range(M)) == 1)
                m1.optimize()
                # print(m1.objVal)
                if m1.objVal > 0:
                    sol = []
                    lhs = gp.LinExpr()
                    rhs = gp.LinExpr()
                    for i in range(M):
                        if z[i].X > 0.5:
                            sol.append(i)
                        if zz[i].X > 0.5:
                            speical = i
                    # print(sol)
                    for i in sol:
                        if i != speical:
                            rhs.add(model._varsy[i])
                        for j in sol:
                            lhs.add(model._varsx[i, j])
                    model.cbCut(lhs <= rhs)
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        valx = model.cbGetSolution(model._varsx)
        selected = gp.tuplelist((i,j) for i,j in model._varsx.keys() if valx[i,j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        #print('inside subtour elim')
        # print('In exact solve, eliminated following subtour:')
        #print(n)
        for asubtour in tour:
            if max(asubtour) < mx:
                lhs = gp.LinExpr()
                for i in asubtour:
                    for j in asubtour:
                        lhs.add(model._varsx[i, j])
                for i in asubtour:
                    rhs = gp.LinExpr()
                    for j in asubtour:
                        if i != j:
                            rhs.add(model._varsy[j])
                    model.cbLazy(lhs <= rhs)

def subtour(edges):
    unvisited = [i for (i,j) in edges]
    cycle = []# initial length has 1 more city
    while unvisited: # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current,'*') if j in unvisited]
        cycle.append(thiscycle)
    return cycle

def ip_solver_st(Prize_set, inner_set, inner_real_set, outer_set, dist_set, alpha_set):
    sPrize = sum(Prize_set) - len(inner_real_set)
    N = len(Prize_set)
    v = {}
    cnt = N - len(inner_real_set)
    for i in inner_real_set:
        v[i] = cnt
        cnt += 1
    m = gp.Model("m")
    m.setParam('OutputFlag', 0)  # change to 1 to show Gurobi solver output
    m.setParam('TIME_LIMIT', 60)
    x = m.addVars(N, N, vtype=GRB.BINARY, name="x")
    y = m.addVars(N, vtype=GRB.BINARY, name="y")
    m.setObjective(sum(dist_set[i, j] * x[i, j] for i in range(N) for j in range(N)),
                   GRB.MINIMIZE)
    m.addConstrs(x.sum(i, '*')  == y[i] for i in range(N))
    m.addConstrs(x.sum(i, '*') - x.sum('*', i) == 0 for i in range(N))
    m.addConstr(sum(x[i, j] for i in inner_set for j in outer_set) == 1)
    m.addConstr(sum(x[i, j] for i in outer_set for j in inner_set) == 1)
    m.addConstr(sum(y[i] for i in range(N - len(inner_real_set), N)) == 1)
    m.addConstrs(y[i] + y[v[i]] <= 1 for i in inner_real_set)
    m.addConstr(y.prod(Prize_set) >= alpha_set*sPrize)
    m.addConstrs(x[i, i] == 0 for i in range(N))
    m._varsx = x
    m._varsy = y
    m.Params.lazyConstraints = 1
    m.optimize(subtourelim)
    vals = m.getAttr('x', x)
    selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
    sol = subtour(selected)[0]
    prize_cd = 0
    it_inner = 0
#     print(sol)
    for i in range(len(sol)):
        prize_cd += Prize_set[sol[i]]
    for i in range(len(sol)-1):
        if sol[i] in inner_set and sol[i+1] in inner_set:
            it_inner += dist_set[sol[i], sol[i+1]]
    if sol[0] in inner_set and sol[-1] in inner_set:
        it_inner += dist_set[sol[-1], sol[0]]
    if m.objVal <= tau:
        it_inner += (tau-m.objVal)
        it_inner_waiting = (tau-m.objVal)
    else:
        it_inner_waiting = 0
    print(m.objVal, prize_cd, it_inner/tau)
    return sol, prize_cd, m.objVal, it_inner/5.2, it_inner_waiting/5.2

In [10]:
data_number = 1
ins = 0
K = 3
tau = 26 * 0.8
with open("HSCPP_real_world_" + str(data_number), "rb") as fp:   # Unpickling
    [Prize_80_vec, dist_80, marker_locations] = pickle.load(fp)

nCusts = 80
x0, y0 = 39.16021, -77.20217 

In [ ]:
dim_alphavec = 11
alphavec = np.linspace(0.7, 1, dim_alphavec)
tau = 26 * 0.8
Nsteps = 15
mx_pc = 0
totalL, totalpc, totalIT, totalIT_wait = [0] * 10, [0] * 10, [0] * 10, [0] * 10
std_IT = [0] * 10
tours = []
prize_cd_vec = []
IT_time_vec = []
L_opt_vec = []
for istep in range(0, 10):
    flag = 0
    Collected_prize = [[0]*dim_alphavec for _ in range(3)]
    tours_rotation = [[0]*dim_alphavec for _ in range(3)]
    IT_time = [[0]*dim_alphavec for _ in range(3)]
    IT_time_waiting = [[0]*dim_alphavec for _ in range(3)]
    IT_time_visit = [[0]*dim_alphavec for _ in range(3)]
    L_vec = [[0]*dim_alphavec for _ in range(3)]
    dist_sys = []
    step = istep
    q1, q2, q3, ix1, ix2, ix3, coord_x, coord_y = separate_quadrant(marker_locations, step)
    l1, l2, l3 = len(q1), len(q2), len(q3)
    ix = [ix1, ix2, ix3]
    inner, inner_real, outer, coord_x, coord_y, Prize, ix = inner_function(q1, q2, q3, ix, Prize_80_vec[ins], 
                                                                           coord_x, coord_y)
    print(l1+len(inner_real[0]), l2+len(inner_real[1]), l3+len(inner_real[2]))
    for ii in range(3):
        dist_current = distance_matrix_quadrant(coord_x[ii], ix[ii], dist_80)
        dist_sys.append(dist_current)
    if min(l1,l2,l3) >= 10 and max(l1,l2,l3) <= 40 and l1 > len(inner_real[0]) and l2 > len(inner_real[1]) and l3 > len(inner_real[2]):
#     print(len(q2),len(inner_real[1])  )
#     if len(q1) > len(inner_real[0]) and len(q2) > len(inner_real[1]) and len(q3) > len(inner_real[2]):
        for ii in range(3):
            for i in range(0, dim_alphavec):
                alpha = alphavec[i]
                mx = len(inner_real[ii]) + len(outer[ii])
                tour, prize_cd, L, IT, IT_inner_waitpt = ip_solver_st(Prize[ii], inner[ii], inner_real[ii], outer[ii], 
                                                                      dist_sys[ii], alpha)
                if L > tau and i==0:
                    flag = 1
                    break
                elif L > tau:
                    break
                tours_rotation[ii][i] = tour
                Collected_prize[ii][i] = prize_cd
                IT_time[ii][i] = IT
                IT_time_waiting[ii][i] = IT_inner_waitpt
                IT_time_visit[ii][i] = IT - IT_inner_waitpt
                L_vec[ii][i] = L
        if flag == 0:
            m = gp.Model("m")
            m.setParam('OutputFlag', 0)  # change to 1 to show Gurobi solver output
            x = m.addVars(3, dim_alphavec, vtype=GRB.BINARY, name="x")
            m.setObjective(sum(Collected_prize[i][j] * x[i, j] for i in range(3) for j in range(dim_alphavec)),
                               GRB.MAXIMIZE)
            m.addConstrs(x.sum(i, '*')  == 1 for i in range(3))
            m.addConstr(sum(IT_time[i][j] * x[i, j] for i in range(3) for j in range(dim_alphavec)) >= 4)
            m.optimize()
            sol = []
            for i in range(3):
                for j in range(dim_alphavec):
                    if x[i,j].X > 0.5:
                        sol.append([i,j])
            if m.objVal > mx_pc:
                mx_pc = m.objVal
                key = step
            for i in range(3):
                totalL[istep] += L_vec[i][sol[i][1]]
                totalpc[istep] += Collected_prize[i][sol[i][1]]
                totalIT[istep] += IT_time[i][sol[i][1]]
                totalIT_wait[istep] += IT_time_waiting[i][sol[i][1]]
            std_IT[istep] = np.std([IT_time[0][sol[0][1]], IT_time[1][sol[1][1]], IT_time[2][sol[2][1]]])
            for ii in range(3):
                tour = tours_rotation[ii][sol[ii][1]].copy()
                for i in range(len(tour)):
                    tour[i] = ix[ii][tour[i]]
                tours.append(tour)
                prize_cd_vec.append(Collected_prize[ii][sol[ii][1]])
                IT_time_vec.append(IT_time[ii][sol[ii][1]])
                L_opt_vec.append(L_vec[ii][sol[ii][1]])
# print(mx_pc, key)

35 31 42
11.699999649999997 11.738287322747993 0.716346170673077
12.2 12.290805560778525 0.6923076923076924
13.0 12.74482633183246 0.6538461538461539
13.4 13.203917354511354 0.6346153846153847
13.799999999999999 13.748185977605589 0.6153846153846155
14.3 14.226466371990817 0.5913461538461539
14.7 14.81099886335813 0.5721153846153847
15.5 15.306846250415608 0.5336538461538463
16.599999999999994 15.745954562139003 0.4807692307692311
18.900000000000006 16.250411489430817 0.37019230769230743
19.999999999999993 16.689519801154212 0.3173076923076927
12.4 10.227391688633137 0.6105769230769231
13.100000000000035 10.655924707476998 0.5624999999999984
14.3 11.327115498170292 0.44711538461538464
15.200000000000003 11.583373193385055 0.40384615384615374
15.8 12.327115498170292 0.44711538461538464
16.100000000000005 12.416948574260138 0.4326923076923075
17.3 12.964596777308175 0.36057692307692313
18.4 13.378841444121502 0.3076923076923078
19.6 13.724932215426112 0.25
22.900000000000002 14.125766381

In [ ]:
def findingskl(tours, dist_80, IT):
    inner_real_all = []
    outer_all = []
    import bisect
    for i in range(80):
        if pos2dist((x0,y0), marker_locations[i]) <= 1.4:
            inner_real_all.append(i)
        else:
            outer_all.append(i)
    time_to_inner_vec = []
    starting_location = [0] * 3
    for i in range(3):
        tours[i].append(tours[i][0])
        for j in range(len(tours[i])-1):
            if tours[i][j] in inner_real_all and tours[i][j+1] in outer_all:
                break
        if j != 0:
            tours[i] = tours[i][j:] + tours[i][1:j]   
        else:
            tours[i] = tours[i][0:-1]   
    for i in range(3):
        tours[i].append(tours[i][0])
        time_to_inner = [dist_80[tours[i][0]][tours[i][1]]]
        for j in range(1, len(tours[i])-1):
            if tours[i][j] in inner_real_all and tours[i][j+1] in inner_real_all:
                break
            time_to_inner.append(dist_80[tours[i][j]][tours[i][j+1]] + time_to_inner[-1])
        time_to_inner_vec.append(time_to_inner)
        if i == 0:
            starting_location[i] = tours[i][j]
        elif i == 1:
            starting_location[i] = tours[i][bisect.bisect_left(time_to_inner, time_to_inner[-1]-IT[0]*5.2)]
        else:
            starting_location[i] = tours[i][0]
    return tours, time_to_inner_vec, starting_location

In [ ]:
with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "tours", "wb") as fp:   # pickling
    pickle.dump(tours, fp)
with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "prize", "wb") as fp:   # pickling
    pickle.dump(prize_cd_vec, fp)
with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "IT", "wb") as fp:   # pickling
    pickle.dump(IT_time_vec, fp)

In [ ]:
# tours, time_to_inner_vec, starting_location = findingskl(tours[0:3], dist_80, IT_time_vec)